In [1]:
import essentia.standard as es
audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"

audio_stereo, sample_rate, num_channels, md5, bit_rate, codec = es.AudioLoader(filename=audio_filename)()
audio_mono = es.MonoLoader(filename=audio_filename, sampleRate=sample_rate)()

import IPython.display as ipd
# ipd.Audio(audio_mono, rate=sample_rate)

In [2]:
import essentia.standard as es
import numpy as np
# audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
audio_filename = "../audio/recorded/guitar_triads.flac"

essentia_audio_stereo, sample_rate, num_channels, md5, bit_rate, codec = es.AudioLoader(filename=audio_filename)()
essentia_audio_mono = es.MonoLoader(filename=audio_filename, sampleRate=sample_rate)()

print(f"Number of channels: {num_channels}")

# convert essentia stereo audio to mono and compare with essentia mono audio
# np.mean(audio, axis=1)

if num_channels == 2:
    audio_mono = np.mean(essentia_audio_stereo, axis=1)
elif num_channels == 1:
    audio_mono = essentia_audio_stereo.T[0]

np.equal(audio_mono, essentia_audio_mono).all()

Number of channels: 1


True

# Tempo

In [3]:
import numpy as np
from scipy.signal import resample

def resample_audio(audio_data: np.ndarray, src_fs: int, target_fs: int) -> np.ndarray:
    """
    Resamples audio data from the source sampling rate to the target sampling rate.

    Parameters:
    audio_data (np.ndarray): The input audio data as a numpy array.
    src_fs (int): The source sampling rate.
    target_fs (int): The target sampling rate.

    Returns:
    np.ndarray: The resampled audio data.
    """
    # Calculate the duration of the audio
    duration = len(audio_data) / src_fs
    # Determine the number of samples for the new sampling rate
    target_num_samples = int(duration * target_fs)
    # Resample the audio data
    resampled_audio = resample(audio_data, target_num_samples)
    return resampled_audio

In [4]:
from essentia.standard import MonoLoader, TempoCNN, Resample
audio_filename = "../audio/recorded/techno_loop.mp3"
audio = MonoLoader(filename=audio_filename, sampleRate=11025, resampleQuality=4)()
audio = MonoLoader(filename=audio_filename)()
# audio = resample_audio(audio, 44100, 11025)
audio = Resample(inputSampleRate=44100, outputSampleRate=11025, quality=4)(audio)
model = TempoCNN(graphFilename="deepsquare-k16-3.pb")
global_tempo, local_tempo, local_tempo_probabilities = model(audio)
global_tempo, local_tempo, local_tempo_probabilities

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `deepsquare-k16-3.pb`
2025-02-08 21:13:09.846872: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


(125.0,
 array([125., 125., 125., 125.], dtype=float32),
 array([0.9201651 , 0.93447894, 0.92712754, 0.93296105], dtype=float32))

In [5]:
import essentia.standard as es
from tempfile import TemporaryDirectory
audio_filename = "../audio/recorded/techno_loop.mp3"

# Loading an audio file.
audio = es.MonoLoader(filename=audio_filename)()

# Compute beat positions and BPM.
rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)

print("BPM:", bpm)
print("Beat positions (sec.):", beats)
print("Beat estimation confidence:", beats_confidence)

BPM: 124.38072967529297
Beat positions (sec.): [ 0.48761904  0.9752381   1.4396372   1.9272562   2.4148753   2.8792744
  3.3668933   3.8545125   4.3189116   4.8065305   5.2941494   5.770159
  6.2461677   6.7337866   7.209796    7.685805    8.173424    8.649433
  9.125442    9.613061   10.08907    10.565079   11.052698   11.5287075
 12.004716   12.492335   12.968345   13.444353   13.9319725  14.407982
 14.88399    15.37161    15.847619   16.323627   16.811247   17.287256
 17.763266   18.250883   18.738503   19.202902   19.690521   20.16653
 20.64254    21.130157   21.606167   22.082176   22.569796   23.057415
 23.521814   24.009432   24.497051   24.96145    25.44907    25.92508
 26.401089   26.888706   27.376326   27.840725   28.328344   28.815964
 29.280363   29.76798    30.2556    ]
Beat estimation confidence: 3.844127655029297


In [6]:
audio_filename = "../audio/recorded/techno_loop.mp3"
# audio_filename = "../audio/recorded/guitar_triads.flac"

audio_mono = es.MonoLoader(filename=audio_filename)()
audio_stereo, sample_rate, num_channels, md5, bit_rate, codec = es.AudioLoader(filename=audio_filename)()

In [7]:
audio_mono

array([-0.00085449, -0.00236511, -0.00366211, ...,  0.00167847,
        0.00187683,  0.00166321], dtype=float32)

In [8]:
audio_stereo, sample_rate, num_channels, md5, bit_rate, codec

(array([[-1.1901855e-03, -5.1879883e-04],
        [-4.8217773e-03,  9.1552734e-05],
        [-5.8593750e-03, -1.4648438e-03],
        ...,
        [ 9.4604492e-04,  2.4108887e-03],
        [ 1.0375977e-03,  2.7160645e-03],
        [ 1.1901855e-03,  2.1362305e-03]], dtype=float32),
 44100.0,
 2,
 '',
 192000,
 'mp3')

In [9]:
audio_mono.shape, audio_stereo.shape

((1354752,), (1354752, 2))

In [10]:
def convert_to_mono(audio: np.ndarray) -> np.ndarray:
    """
    Converts a stereo audio signal to mono by averaging the left and right channels.

    Parameters:
    audio (np.ndarray): The input stereo audio signal.

    Returns:
    np.ndarray: The mono audio signal.
    """
    return np.mean(audio, axis=1)

# Key Estimation
https://essentia.upf.edu/reference/std_KeyExtractor.html

In [11]:
import essentia.standard as es
audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"

audio_stereo, sample_rate, num_channels, md5, bit_rate, codec = es.AudioLoader(filename=audio_filename)()
audio_mono = es.MonoLoader(filename=audio_filename, sampleRate=sample_rate)()

print(f"Infile: {audio_filename}")
print(f"Sample rate: {sample_rate}")
print(f"Number of channels: {num_channels}")
print(f"MD5 checksum: {md5}")
print(f"Bit rate: {bit_rate}")
print(f"Codec: {codec}")

Infile: ../audio/recorded/mozart_c_major_30sec.wav
Sample rate: 44100.0
Number of channels: 2
MD5 checksum: 
Bit rate: 1411200
Codec: pcm_s16le


In [12]:
import IPython.display as ipd
# ipd.Audio(audio_mono, rate=sample_rate)

In [13]:
key_temperley = es.KeyExtractor(profileType='temperley')
key_krumhansl = es.KeyExtractor(profileType='krumhansl')
key_edma = es.KeyExtractor(profileType='edma')

# key, scale, strength = es.KeyExtractor(profileType="temperley")(audio_mono)
print("Key (Temperley):", key_temperley(audio_mono))
print("Key (Krumhansl):", key_krumhansl(audio_mono))
print("Key (Edma):", key_edma(audio_mono))

Key (Temperley): ('C', 'major', 0.9181052446365356)
Key (Krumhansl): ('C', 'major', 0.9224876761436462)
Key (Edma): ('C', 'major', 0.8909444808959961)


# LoudnessEBUR128

In [14]:
import essentia.standard as es
audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
audio_filename = "../audio/recorded/techno_loop.mp3"

audio_stereo, sample_rate, num_channels, md5, bit_rate, codec = es.AudioLoader(filename=audio_filename)()
audio_mono = es.MonoLoader(filename=audio_filename, sampleRate=sample_rate)()

print(f"Infile: {audio_filename}")
print(f"Sample rate: {sample_rate}")
print(f"Number of channels: {num_channels}")
print(f"MD5 checksum: {md5}")
print(f"Bit rate: {bit_rate}")
print(f"Codec: {codec}")

loundsEBUR128 = es.LoudnessEBUR128(sampleRate=sample_rate)

momentaryLoudness, shortTermLoudness, integratedLoudness, loudnessRange = loundsEBUR128(audio_stereo)
print(f"Integrated loudness: {integratedLoudness:.2f} LUFS")
print(f"Loudness range: {loudnessRange:.2f} LU")
print(f"Short-term loudness: {shortTermLoudness.shape} values")
print(f"Momentary loudness: {momentaryLoudness.shape} values")

Infile: ../audio/recorded/techno_loop.mp3
Sample rate: 44100.0
Number of channels: 2
MD5 checksum: 
Bit rate: 192000
Codec: mp3
Integrated loudness: -13.12 LUFS
Loudness range: 0.54 LU
Short-term loudness: (279,) values
Momentary loudness: (305,) values


In [15]:
print(type(audio_stereo), type(audio_mono))
print(audio_stereo.shape, audio_mono.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1354752, 2) (1354752,)


In [16]:
import essentia.standard as es
from utils import load_audio

audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

# Mono Algorithm
rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio_mono)

print("BPM:", bpm)

# Stereo Algorithm
loundsEBUR128 = es.LoudnessEBUR128(sampleRate=sample_rate)

momentaryLoudness, shortTermLoudness, integratedLoudness, loudnessRange = loundsEBUR128(audio_stereo)
print(f"Integrated loudness: {integratedLoudness:.2f} LUFS")
print(f"Loudness range: {loudnessRange:.2f} LU")
print(f"Short-term loudness: {shortTermLoudness.shape} values")
print(f"Momentary loudness: {momentaryLoudness.shape} values")

print(type(audio_stereo), type(audio_mono))
print(audio_stereo.shape, audio_mono.shape)


BPM: 154.6952667236328
Integrated loudness: -10.36 LUFS
Loudness range: 3.49 LU
Short-term loudness: (271,) values
Momentary loudness: (297,) values
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1323000, 2) (1323000,)


# Embeddings

## Discogs-EffNet
https://essentia.upf.edu/models.html#discogs-effnet

## MSD-MusiCNN
https://essentia.upf.edu/models.html#msd-musicnn

In [17]:
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs

audio = MonoLoader(filename="../audio/recorded/mozart_c_major_30sec.wav", sampleRate=16000, resampleQuality=4)()
model = TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
effnet_embeddings = model(audio)
print(effnet_embeddings.shape)

from essentia.standard import MonoLoader, TensorflowPredictMusiCNN

# audio = MonoLoader(filename="audio.wav", sampleRate=16000, resampleQuality=4)()
model = TensorflowPredictMusiCNN(graphFilename="msd-musicnn-1.pb", output="model/dense/BiasAdd")
musicnn_embeddings = model(audio)
print(musicnn_embeddings.shape)

(29, 1280)
(19, 200)


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`


In [18]:
import essentia.standard as es
from utils import load_audio

audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

resampler = es.Resample(inputSampleRate=sample_rate, outputSampleRate=16000, quality=4)
resampled_audio_for_model = resampler(audio_mono)

# Effnet Model
model = es.TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
my_effnet_embeddings = model(resampled_audio_for_model)
print(my_effnet_embeddings.shape)

# Musicnn Model
model = es.TensorflowPredictMusiCNN(graphFilename="msd-musicnn-1.pb", output="model/dense/BiasAdd")
my_musicnn_embeddings = model(resampled_audio_for_model)
print(my_musicnn_embeddings.shape)


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`


(29, 1280)
(19, 200)


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`


In [19]:
import numpy as np
print(np.equal(effnet_embeddings, my_effnet_embeddings).all())
print(np.equal(musicnn_embeddings, my_musicnn_embeddings).all())


True
True


# Music genre and style
https://essentia.upf.edu/models.html#genre-discogs400

In [20]:
import numpy as np
import essentia.standard as es
from utils import load_audio


# audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
# audio_filename = "../audio/recorded/guitar_triads.flac"
audio_filename = "../audio/recorded/techno_loop.mp3"

audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

resampler = es.Resample(inputSampleRate=sample_rate, outputSampleRate=16000, quality=4)
resampled_audio_for_model = resampler(audio_mono)


embedding_model = es.TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
embeddings = embedding_model(resampled_audio_for_model)

model = es.TensorflowPredict2D(graphFilename="genre_discogs400-discogs-effnet-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall:0")
predictions = model(embeddings)
print(predictions.shape)

# play audio 
import IPython.display as ipd
# ipd.Audio(audio_mono, rate=sample_rate)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`


(29, 400)


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`


In [21]:
import pandas as pd
pd.DataFrame(predictions).describe()

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
count,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,...,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01,2.900000e+01
mean,1.798274e-07,3.803591e-08,2.631822e-10,1.074023e-09,5.713444e-10,8.829512e-08,7.170690e-09,5.478491e-10,2.108455e-08,3.715666e-09,...,2.623119e-07,8.099414e-09,4.837260e-09,3.621174e-08,3.768234e-10,1.777238e-08,6.198116e-09,2.453617e-07,1.579460e-06,2.020852e-07
std,1.300585e-07,2.562721e-08,2.139434e-10,9.062933e-10,5.180064e-10,5.046985e-08,4.778453e-09,3.716845e-10,1.247041e-08,2.114510e-09,...,1.006252e-07,3.155751e-09,3.469908e-09,2.850072e-08,3.179635e-10,1.145881e-08,3.212875e-09,9.302547e-08,7.380336e-07,9.481791e-08
min,6.932350e-08,6.792547e-09,4.170675e-11,1.529156e-10,8.821727e-11,1.876821e-08,1.603472e-09,8.369673e-11,3.898750e-09,1.319297e-09,...,1.182848e-07,4.056866e-09,1.233375e-09,7.713763e-09,6.477409e-11,3.819610e-09,1.545011e-09,1.377783e-07,6.384285e-07,6.745645e-08
25%,1.029128e-07,1.809654e-08,9.881071e-11,4.148787e-10,2.428103e-10,5.108252e-08,5.066071e-09,2.012496e-10,1.141905e-08,1.964045e-09,...,1.953179e-07,5.592237e-09,3.023485e-09,1.867690e-08,1.627591e-10,8.182335e-09,3.792763e-09,1.701085e-07,1.205519e-06,1.104678e-07
50%,1.390289e-07,3.382349e-08,1.836513e-10,8.281908e-10,4.647272e-10,8.330538e-08,6.228497e-09,5.566015e-10,1.770760e-08,3.048536e-09,...,2.466960e-07,7.621918e-09,4.228390e-09,3.021342e-08,2.895602e-10,1.525999e-08,6.299778e-09,2.214135e-07,1.387036e-06,2.062225e-07
75%,2.067729e-07,4.750502e-08,3.849712e-10,1.269661e-09,6.396383e-10,1.186228e-07,9.158888e-09,7.967885e-10,3.086538e-08,5.133899e-09,...,3.409746e-07,9.397820e-09,5.573698e-09,3.985556e-08,4.922110e-10,2.521168e-08,7.460681e-09,2.936793e-07,1.904347e-06,2.683934e-07
max,7.306193e-07,1.249576e-07,8.249196e-10,4.013857e-09,2.663589e-09,2.350246e-07,2.291770e-08,1.374038e-09,4.954006e-08,1.002066e-08,...,5.203691e-07,1.886137e-08,1.824901e-08,1.530150e-07,1.587943e-09,5.644729e-08,1.377729e-08,4.764748e-07,3.727350e-06,4.362982e-07


In [4]:
import json

def load_classes(filename: str) -> dict:
    with open(filename, "r") as f:
        classes = json.load(f)
    return classes

genre_classes = load_classes("genre_discogs400-discogs-effnet-1.json")

In [23]:
len(genre_classes["classes"])

400

In [24]:
import numpy as np

# predictions: shape (num_seconds, num_classes)

# sum or average across time
avg_probs = np.mean(predictions, axis=0)

# normalize to ensure it sums to 1
overall_probs = avg_probs / np.sum(avg_probs)

# print("Overall probabilities: ", overall_probs)
# print("Class with highest probability:", np.argmax(overall_probs))

In [25]:
highest_prob = np.max(overall_probs)
predicted_class = genre_classes["classes"][np.argmax(overall_probs)]
print(f"Predicted class '{predicted_class}' with probability {highest_prob:.2f}")

Predicted class 'Electronic---Dub Techno' with probability 0.32


In [26]:
import matplotlib.pyplot as plt
import numpy as np

class_labels = genre_classes["classes"]  # e.g., ['class_a', 'class_b', 'class_c', 'class_d']

# 1) Sort indices by descending probabilities
sorted_indices = np.argsort(overall_probs)[::-1]

# 2) Take the top 10 indices (or fewer if you don't have that many classes)
top10_indices = sorted_indices[:10]

# 3) Slice the probabilities and labels for those top 10
top10_probs = overall_probs[top10_indices]
top10_labels = [class_labels[i] for i in top10_indices]

# # Plot
# plt.figure(figsize=(8, 5))
# plt.bar(range(len(top10_probs)), top10_probs, tick_label=top10_labels)
# plt.title("Top 10 Classes by Probability")
# plt.xlabel("Classes")
# plt.ylabel("Probability")
# plt.ylim([0, 1])  # probabilities range between 0 and 1
# plt.xticks(rotation=45)  # rotate labels if they're long
# plt.tight_layout()
# plt.show()

# Voice/instrumental classifier
https://essentia.upf.edu/models.html#voice-instrumental

In [27]:
voice_instrumet_classes = load_classes("voice_instrumental-audioset-vggish-1.json")
voice_instrumet_classes["classes"]

['instrumental', 'voice']

In [28]:
import numpy as np
import essentia.standard as es
from utils import load_audio
import IPython.display as ipd


audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
# audio_filename = "../audio/recorded/guitar_triads.flac"
# audio_filename = "../audio/recorded/techno_loop.mp3"

audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

ipd.Audio(audio_mono, rate=sample_rate)

resampler = es.Resample(inputSampleRate=sample_rate, outputSampleRate=16000, quality=4)
resampled_audio_for_model = resampler(audio_mono)

embedding_model = es.TensorflowPredictVGGish(graphFilename="audioset-vggish-3.pb", output="model/vggish/embeddings")
embeddings = embedding_model(resampled_audio_for_model)

model = es.TensorflowPredict2D(graphFilename="voice_instrumental-audioset-vggish-1.pb", output="model/Softmax")
predictions = model(embeddings)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `audioset-vggish-3.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `voice_instrumental-audioset-vggish-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `voice_instrumental-audioset-vggish-1.pb`


In [29]:
import numpy as np

def get_probabilities(predictions, genre_classes):

    # Average probabilities across time
    avg_probs = np.mean(predictions, axis=0)
    
    # Normalize so they sum to 1
    overall_probs = avg_probs / np.sum(avg_probs)
    
    # Build a dictionary: {class_name: probability}
    prob_dict = {
        genre_classes["classes"][i]: overall_probs[i]
        for i in range(len(genre_classes["classes"]))
    }
    return prob_dict

def get_highest_probability(prob_dict):
    # Find the class with the highest probability
    best_class = max(prob_dict, key=prob_dict.get)
    return best_class, prob_dict[best_class]

In [30]:
predictions = get_probabilities(predictions, voice_instrumet_classes)
predictions

{'instrumental': 0.9675827, 'voice': 0.032417323}

In [31]:
get_highest_probability(predictions)

('instrumental', 0.9675827)

# Danceability



In [32]:
danceability_classes = load_classes("danceability-audioset-vggish-1.json")
danceability_classes["classes"]

['danceable', 'not_danceable']

In [33]:
import numpy as np
import essentia.standard as es
from utils import load_audio
import IPython.display as ipd


# audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
# audio_filename = "../audio/recorded/guitar_triads.flac"
audio_filename = "../audio/recorded/techno_loop.mp3"

audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

ipd.Audio(audio_mono, rate=sample_rate)

resampler = es.Resample(inputSampleRate=sample_rate, outputSampleRate=16000, quality=4)
resampled_audio_for_model = resampler(audio_mono)

embedding_model = es.TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
embeddings = embedding_model(resampled_audio_for_model)

model = es.TensorflowPredict2D(graphFilename="danceability-discogs-effnet-1.pb", output="model/Softmax")
predictions = model(embeddings)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `danceability-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `danceability-discogs-effnet-1.pb`


In [34]:
get_probabilities(predictions, danceability_classes)

{'danceable': 0.99933267, 'not_danceable': 0.000667325}

# Arousal/valence emoMusic
https://essentia.upf.edu/models.html#arousal-valence-emomusic

In [5]:
arousal_valence_classes = load_classes("emomusic-msd-musicnn-2.json")
arousal_valence_classes["classes"]

['valence', 'arousal']

In [2]:
import numpy as np
import essentia.standard as es
from utils import load_audio
import IPython.display as ipd


audio_filename = "../audio/recorded/mozart_c_major_30sec.wav"
# audio_filename = "../audio/recorded/guitar_triads.flac"
# audio_filename = "../audio/recorded/techno_loop.mp3"

audio_data = load_audio(audio_filename)
audio_stereo = audio_data.audio_stereo
audio_mono = audio_data.audio_mono
sample_rate = audio_data.sample_rate

resampler = es.Resample(inputSampleRate=sample_rate, outputSampleRate=16000, quality=4)
resampled_audio_for_model = resampler(audio_mono)

embedding_model = es.TensorflowPredictMusiCNN(graphFilename="msd-musicnn-1.pb", output="model/dense/BiasAdd")
embeddings = embedding_model(resampled_audio_for_model)

model = es.TensorflowPredict2D(graphFilename="emomusic-msd-musicnn-2.pb", output="model/Identity")
predictions = model(embeddings)
# get_probabilities(predictions, arousal_valence_classes)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `msd-musicnn-1.pb`
2025-02-12 00:50:41.426135: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `emomusic-msd-musicnn-2.pb`


In [7]:
prds = np.mean(predictions, axis=0)
prds

array([4.6252866, 4.5481224], dtype=float32)

In [6]:
a,b = prds
a, b

(4.6252866, 4.5481224)